In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

def scrape_acha_dynamic_stats():
    firefox_options = Options()
    firefox_options.add_argument("--headless")  

    geckodriver_path = r"C:\Program Files\GeckoDriver\geckodriver.exe"
    service = Service(executable_path=geckodriver_path)

    with webdriver.Firefox(service=service, options=firefox_options) as driver:
        url = "https://www.achahockey.org/stats/standings?season=46&conference=10&division=-1&standingstype=division&context=overall&specialteams=false&sortkey=points&league=1"
        driver.get(url)

        all_data = []
        
        while True:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
            
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            
            table = soup.find('table')
            if table:
                rows = table.find_all('tr')
                for row in rows[1:]:  # Skip header row
                    cols = row.find_all('td')
                    cols = [ele.text.strip() for ele in cols]
                    all_data.append(cols)

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Next')]"))
                )
                next_button.click()
                time.sleep(2)
            except:
                break

    return all_data

acha_dynamic_stats = scrape_acha_dynamic_stats()

column_names = ['Team', 'GP', 'W', 'L', 'OTL', 'T', 'SOW', 'SOL', 'PTS', 'OTW', 'GF', 'GA', 'DIFF', 'PCT', 'PCT2', 'PIM', 'RW', 'ROW', 'STK', 'IN-DIV', 'GPCT', 'P10', 'Nickname']


data_without_first_two = [row[2:] for row in acha_dynamic_stats]

df = pd.DataFrame(data_without_first_two, columns=column_names)


int_columns = ['RW', 'ROW', 'GP', 'W', 'L', 'OTL', 'T', 'SOW', 'SOL', 'PTS', 'OTW', 'GF', 'GA', 'DIFF', 'PIM']
for column in int_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0).astype(int)


float_columns = ['PCT', 'GPCT', 'PCT2']
for column in float_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')


string_columns = ['Team', 'STK', 'IN-DIV', 'P10', 'Nickname']
for column in string_columns:
    df[column] = df[column].astype(str)


df = df[df['Team'] != 'None']
df = df.dropna(subset=['Team'])

print(df)

print("\nDataFrame Info:")
print(df.info())

df.to_csv('acha_md3_team.csv', index=False)

      Team  GP  W   L  OTL  T  SOW  SOL  PTS  OTW  ...    PCT  PCT2  PIM  RW  \
0    M3WCU  10  4   6    0  0    0    0    8    0  ...  0.400   0.0  124   4   
1     YCPA  13  4   9    0  0    0    0    8    0  ...  0.308   0.0  128   4   
2    GMUM3   6  1   5    0  0    0    0    2    0  ...  0.167   0.0   53   1   
3     STVU   6  1   5    0  0    0    0    2    0  ...  0.167   0.0   76   1   
4    LOYUM   0  0   0    0  0    0    0    0    0  ...  0.000   0.0    0   0   
..     ...  .. ..  ..  ... ..  ...  ...  ...  ...  ...    ...   ...  ...  ..   
121    UCD  11  5   4    1  1    0    0   12    0  ...  0.545   0.0    6   5   
122   M3PU  15  2  11    1  0    0    1    6    0  ...  0.200   0.0  208   2   
123  M3USD   9  1   8    0  0    0    0    2    0  ...  0.111   0.0   49   1   
124   UCSC   5  0   5    0  0    0    0    0    0  ...  0.000   0.0   51   0   
125   CALU  10  0  10    0  0    0    0    0    0  ...  0.000   0.0   40   0   

     ROW       STK   IN-DIV   GPCT     